In [4]:
using Pkg
Pkg.activate(@__DIR__)
push!(LOAD_PATH, joinpath(@__DIR__, "..", ".."))
Pkg.instantiate()

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


In [5]:
using JuMP
using BARON
using LinearAlgebra
using StaticArrays
using MINLPTrajOpt
using MINLPTrajOpt.SO3
using Rotations
using Random

┌ Info: Recompiling stale cache file /home/twan/.julia/compiled/v1.0/MINLPTrajOpt/myGxE.ji for MINLPTrajOpt [c68a6224-cbe5-11e8-1e7f-b1d3117b2aa4]
└ @ Base loading.jl:1184


In [6]:
rootdir = joinpath(dirname(pathof(MINLPTrajOpt)), "..")
save_data = false;

In [7]:
solver = BaronSolver(threads=Sys.CPU_THREADS, EpsA=1e-2)
# solver = CouenneNLSolver()
parameters = SO3Parameters()
τmax = 10.0
Random.seed!(1)
quat0 = rand(Quat)
# ω0 = SVector(0.1, 0.2, 0.3)
ω0 = zero(SVector{3})

N = 7
T = 4.0
Δt = T / N
Δθmax = 0.5

problem = SO3Problem(parameters, quat0, ω0, solver; τmax=τmax, Δt=Δt, N=N, Δθmax=Δθmax);

In [8]:
status = solve(problem.model)
@assert status == :Optimal

 BARON version 18.8.23. Built: LNX-64 Thu Aug 23 14:46:44 EDT 2018

 BARON is a product of The Optimization Firm.
 For information on BARON, see https://minlp.com/about-baron
 Licensee: MIT at Twan Koolen, tkoolen@mit.edu.

 If you use this software, please cite publications from
 https://minlp.com/baron-publications, such as: 

 Khajavirad, A. and N. V. Sahinidis,
 A hybrid LP/NLP paradigm for global optimization relaxations,
 Mathematical Programming Computation, 10, 383-421, 2018.
 This BARON run may utilize the following subsolver(s)
 For LP/MIP: ILOG CPLEX                                      
 For NLP: IPOPT, FILTERSD, FILTERSQP
 Doing local search
 Unable to find/load CPLEX library libcplex.so: cannot open shared object file: No such file or directory
 Using CPLEX library libcplex1280.so.
 Preprocessing found feasible solution with value  1.61296223848     
 Solving bounding LP
 Starting multi-start local search
 Done with local search
  Iteration    Open nodes         Time (s) 

In [9]:
if save_data
    i = 0
    while (datadir = joinpath(rootdir, "data", "so3", "run$i"); isdir(datadir))
        i += 1
    end
    mkpath(datadir)
end
if save_data && solver isa BaronSolver
    cp(dirname(problem.model.internalModel.options[:SumName]), joinpath(datadir, "baronfiles"))
end

In [10]:
dirname(problem.model.internalModel.options[:SumName])

"/tmp/tmp0IzM1d"

In [11]:
t = range(0, T, length=N+1)
quats = getvalue.(problem.q)
pushfirst!(quats, problem.q0)
qs = [[quat.w, quat.x, quat.y, quat.z] for quat in quats]
Δquats = getvalue.(problem.Δq)
ωs = getvalue.(problem.ω)
ωds = getvalue.(problem.ωd)
τs = getvalue.(problem.τ);

In [12]:
using Test
for Δquat in Δquats
    Δθ = 2 * acos(Δquat.w)
    @test Δθ <= Δθmax + 1e-6
    @test norm([Δquat.w, Δquat.x, Δquat.y, Δquat.z]) ≈ 1 atol=1e-6
end

In [13]:
using RigidBodyDynamics

In [14]:
world = RigidBody{Float64}("world")
mechanism = Mechanism(world)
joint = Joint("joint", QuaternionSpherical{Float64}())
body = RigidBody{Float64}("body")
attach!(mechanism, world, body, joint);

In [15]:
using MeshCatMechanisms, MechanismGeometries, CoordinateTransformations, ColorTypes, GeometryTypes

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/25f9570346b69729e3f0d089746606f6943c897a-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/25f9570346b69729e3f0d089746606f6943c897a-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [33]:
struct SO3Geometry <: MechanismGeometries.AbstractGeometrySource end
function MechanismGeometries.visual_elements(mechanism::Mechanism, source::SO3Geometry)
    @assert length(joints(mechanism)) == 1
    joint = first(joints(mechanism))
    frame = frame_after(joint)
    elements = Vector{VisualElement}()
    boxdims = Vec(0.3, 0.3, 0.3)
    box = HyperRectangle(-boxdims / 2, boxdims)
    push!(elements, VisualElement(frame, box, MechanismGeometries.DEFAULT_COLOR, IdentityTransformation()))
    elements
end

In [34]:
mvis = MechanismVisualizer(mechanism, SO3Geometry());

In [35]:
open(mvis);

┌ Info: Serving MeshCat visualizer at http://127.0.0.1:8704
└ @ MeshCat /home/twan/.julia/packages/MeshCat/rjDJh/src/servers.jl:12
┌ Info: Listening on: Sockets.InetAddr{Sockets.IPv4}(ip"127.0.0.1", 0x2200)
└ @ HTTP.Servers /home/twan/.julia/packages/HTTP/nUK4f/src/Servers.jl:307
┌ Info: Accept:  🔗    0↑     0↓    0s 127.0.0.1:8704:8704 ≣16
└ @ HTTP.Servers /home/twan/.julia/packages/HTTP/nUK4f/src/Servers.jl:366
┌ Info: Accept:  🔗    0↑     0↓    0s 127.0.0.1:8704:8704 ≣16
└ @ HTTP.Servers /home/twan/.julia/packages/HTTP/nUK4f/src/Servers.jl:366
┌ Info: Accept:  🔗    0↑     0↓    0s 127.0.0.1:8704:8704 ≣16
└ @ HTTP.Servers /home/twan/.julia/packages/HTTP/nUK4f/src/Servers.jl:366
Error handling websocket connection:
WebSockets.WebSocketClosedError("ws|server respond to OPCODE_CLOSE 1001:Going Away")┌ Info: Closed:  💀    1↑     1↓🔒  24s 127.0.0.1:8703:8703 ≣16
└ @ HTTP.Servers /home/twan/.julia/packages/HTTP/nUK4f/src/Servers.jl:373


In [38]:
setanimation!(mvis, t, qs)

┌ Info: Closed:  💀    1↑     1↓🔒 115s 127.0.0.1:8702:8702 ≣16
└ @ HTTP.Servers /home/twan/.julia/packages/HTTP/nUK4f/src/Servers.jl:373
┌ Info: Closed:  💀    4↑     4↓🔒 115s 127.0.0.1:8702:8702 ≣16
└ @ HTTP.Servers /home/twan/.julia/packages/HTTP/nUK4f/src/Servers.jl:373


In [20]:
τs

7-element Array{Array{Float64,1},1}:
 [-0.204157, 0.319071, 0.00557661]    
 [0.0, 0.0, 0.0]                      
 [0.0, 0.0, 0.0]                      
 [0.0296464, -0.0463334, -0.000809799]
 [0.0662166, -0.103488, -0.00180872]  
 [0.050399, -0.078767, -0.00137666]   
 [0.0230396, -0.0360078, -0.000629332]